# Loading raw video

In [2]:
import numpy as np
import cv2
import os
import glob
import math

In [20]:
labeled_videos = glob.glob('./labeled/*.hevc')
video_index = 4

cap = cv2.VideoCapture(labeled_videos[video_index])

frame_count = 0

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if(ret == False):
        break
    else:
        cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_count += 1
    
cap.release()
cv2.destroyAllWindows()

print(frame_count)

1196


 # Visualising GT pitch and yaw (respectively in 2D array)

In [19]:
def draw_gaze(image_in, cam_pos, pitchyaw, length=40.0, thickness=2, color=(255, 0, 0)):
    """Draw gaze angle on given image with a given cam positions."""
    image_out = image_in
    if len(image_out.shape) == 2 or image_out.shape[2] == 1:
        image_out = cv2.cvtColor(image_out, cv2.COLOR_GRAY2BGR)
    dx = -length * np.sin(pitchyaw[1])
    dy = -length * np.sin(pitchyaw[0])
    cv2.arrowedLine(image_out, tuple(np.round(cam_pos).astype(np.int32)),
                   tuple(np.round([cam_pos[0] + dx, cam_pos[1] + dy]).astype(int)), color,
                   thickness, cv2.LINE_AA, tipLength=0.3)
    return image_out

In [31]:
labeled_videos = glob.glob('./labeled/*.hevc')
gt_pitch_yaw = glob.glob('./labeled/*.txt')
video_index = 1


pitch_yaw = []


with open(gt_pitch_yaw[video_index], 'r') as f:
    for line in f.readlines():
        pitch_yaw.append(float(x) for x in line.split(' '))

#frame size (874, 1164, 3)
cap = cv2.VideoCapture(labeled_videos[video_index])
frame_count = 0


while(cap.isOpened()):
    ret, frame = cap.read()
    try:
        current_p_y = list(pitch_yaw[frame_count])
    except:
        next
    
    if(ret == False):
        break
    else:
        pitch = 'Pitch: ' + str(current_p_y[0])
        yaw = 'Yaw:  ' + str(current_p_y[1])
        cv2.putText(frame, pitch, (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 1, 2)
        cv2.putText(frame, yaw, (20,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 1, 2)
        
        if(not math.isnan(current_p_y[0])):
            draw_gaze(frame, (582, 850), current_p_y, length=3000, thickness = 15)
            draw_gaze(frame, (582, 850), (0,0), length=3000, thickness = 15, color = (255,255,255)) #reference, perfect calib
        cv2.imshow('frame',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_count += 1
    
cap.release()
cv2.destroyAllWindows()

print(frame_count)

1200
